In [1]:
from IPython.display import HTML
def iframe(url):
    iframe = '<iframe src=' + url + ' width=700 height=350></iframe>'
    return(HTML(iframe))

# folium <small>Python Data. Leaflet.js Maps.</small> 

Static maps, like the sort you'd develop for a publication or brochure, are worthwhile for their durability, flexibility, and relative ease in creation. However, in a time where the creation of public-facing research products is demanded by an increasing range of domains (& funding sources), being able to create interactive, web-based displays of geospatial data is critical. 

There are myriad ways to approach this challenge. Hard-core web developers can roll their own websites from scratch in Javascript, HTML, and CSS, leveraging their own data APIs and maintaining their own cyberinfrastructure. Hard-core domain researchers can simply put some easy-to-read plots on a simple website and call it a day. Somewhere in between lies the ability to take an already-polished data analysis pipeline in Python and output the geospatial results to a web-friendly map ready to be hosted on a simple static hosting solution (like your departmental web server or Amazon S3). 

That's the intent of the Python package [`folium`](https://github.com/python-visualization/folium): to combine data objects in Python with a web mapping framework known as Leaflet to produce interactive geospatial data products on the web. 

## Table of Contents:

1. [Folium Basics](#Folium-Basics)
2. [Markers and Polygons](#Markers-and-Polygons)
3. [Shapefiles](#Shapefiles)
4. [Specialized Basemaps](#Specialized-Basemaps)
5. [Synthesis Example: River Flood Stage](#Synthesis-Example:-River-Flood-Stage)
6. [Synthesis Example: Choropleth Maps](#Choropleth-Maps)

## Folium Basics

It's really easy to create a folium map and save it to a self-contained HTML file, which can then be uploaded to a web server somewhere and shown to the world. 

In [2]:
import folium
MAP_CENTER = [46.852947, -121.760424] ## Mt. Rainier
m = folium.Map(location=MAP_CENTER, zoom_start=6) ## start at zoom level 6.
m.save('my-first-map.html')

That's it. Click [here](my-first-map.html) to see the map you just made. Yep, you made that one. I'll use a helper utility that I wrote above to embed that HTML page into our current notebook for viewing (this won't work if you don't run the cell above).

In [3]:
iframe('my-first-map.html')

Alternatively, since the object `m` is the Folium map that we created, we can just display it natively in the notebook itself. We'll do this from now on, but I wanted to demonstrate that you can create a HTML file containing your interactive map that you can zoom and pan around in 3 lines of Python. 

In [4]:
m = folium.Map(location=MAP_CENTER, zoom_start=6) ## start at zoom level 10.
m

## Markers and Polygons

The simplest mechanism that Folium has to add data to maps is the `Marker` class. It's about as simple as it looks. 

In [5]:
m = folium.Map(MAP_CENTER, zoom_start=11, tiles='Stamen Terrain')
folium.Marker([46.8354, -121.7325], popup='Camp Muir').add_to(m)
m

If you click on the marker, you'll get a popup with `Camp Muir` in it (one of the climber's camps on the Dissapointment Cleaver route to the summit of Mt. Rainier). The markers are also fairly customizable:

In [6]:
folium.Marker([46.852947, -121.760424],
              icon=folium.Icon(icon='thumbs-up', color='red'), 
              popup='Mt.Rainier Summit').add_to(m)

folium.Marker([46.7854178498, -121.734817028],
              icon=folium.Icon(icon='home', color='green'),
              popup='Paradise').add_to(m)

m

## Shapefiles

As of 31 August 2017, the only way to specify geometries to Folium is through the GeoJSON format. This is a little strange if you're experienced with Geopandas, but fortunately those who are comfortable using Geopandas can perform their geospatial analyses in Geopandas and use the `to_json()` function on GeoDataFrames to return the GeoJSON representation. In this way we can use Geopandas and Folium together to display other kinds of more complex geometries on Leaflet interactive maps.

We'll download a shapefile of the Disappointment Cleaver route on Mt. Rainier and use Geopandas to put it on the map. 

In [9]:
import geopandas as gpd
dc_route = gpd.read_file("../data/DC_shp/")
mappable = folium.features.GeoJson(dc_route.to_json())
mappable.add_child(folium.Popup('DC Route')).add_to(m)
m

## Specialized Basemaps
By default, Folium uses [OpenStreetMap](http://www.openstreetmap.org/) basemaps, which are good for most purposes. However, there are other options, which you can find on the Folium [github page](https://github.com/python-visualization/folium). To employ one of these specialized basemaps, all you need to do is specify the `tiles=` keyword argument to the `folium.Map` function:

In [10]:
folium.Map(MAP_CENTER, zoom_start=6, tiles='Mapbox Control Room')

Folium can also use Mapbox custom tilesets with `tiles='Mapbox'` and `api_key=[your tileset api key]` as arguments to `folium.Map`. Finally, folium is able to use *any custom tileset*, provided there's a URL to access it: 

```
tileset = r'http://{s}.tiles.yourtiles.com/{z}/{x}/{y}.png'
map = folium.Map(location=[45.372, -121.6972], zoom_start=12,
                 tiles=tileset, attr='My Data Attribution')
```

## Synthesis Example: River Flood Stage

The US National Weather Service provides shapefiles of about ~8500 river stage gauges around the country, updated every 15 minutes. We'll download the current data and create a folium map with the gauges in flood stage, complete with the image of their hydrograph in a popup. We start by downloading the data. 

In [11]:
! curl -L https://water.weather.gov/ahps/download.php?data=tgz_obs > national_shapefile_obs.tgz
! tar xvzf national_shapefile_obs.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1007k  100 1007k    0     0   550k      0  0:00:01  0:00:01 --:--:-- 2178k
national_shapefile_obs.shp
national_shapefile_obs.shx
national_shapefile_obs.dbf
national_shapefile_obs.prj
national_shapefile_obs.shp.xml


In [12]:
riverdata = gpd.read_file("national_shapefile_obs.shp")
print("%d stations in file" % len(riverdata))

8554 stations in file


We'll take a look at what we've got:

In [13]:
riverdata.head()

,GaugeLID,Status,Location,Latitude,Longitude,Waterbody,State,Observed,ObsTime,Units,...,Major,LowThresh,LowThreshU,WFO,HDatum,PEDTS,SecValue,SecUnit,URL,geometry
0,AAKA2,no_flooding,Auke Bay,58.851944,-134.708611,Antler River,AK,32.05,2017-09-08 21:30:00,ft,...,,,ft,pajk,NAD83/WGS84,HGIRZ,0.45,kcfs,https://water.weather.gov/ahps2/hydrograph.php...,POINT (-134.708611 58.851944)
1,AAMC1,not_defined,Alameda tide gage,37.771667,-122.298333,San Francisco Bay,CA,6.67,2017-09-08 21:48:00,ft,...,,,ft,mtr,NAD83/WGS84,HMIRG,-999.00,kcfs,https://water.weather.gov/ahps2/hydrograph.php...,POINT (-122.298333 37.771667)
2,AANG1,no_flooding,Atlanta,33.819444,-84.407778,Peachtree Creek,GA,2.31,2017-09-08 21:00:00,ft,...,20.00,,ft,ffc,NAD83/WGS84,HGIRG,0.01,kcfs,https://water.weather.gov/ahps2/hydrograph.php...,POINT (-84.40777799999999 33.819444)
3,ABAG1,no_flooding,GA Highway 203 near Blackshear,31.375278,-82.288889,Alabaha River,GA,3.06,2017-09-08 21:30:00,ft,...,14.00,1.60,ft,jax,NAD83/WGS84,HGIRG,0.04,kcfs,https://water.weather.gov/ahps2/hydrograph.php...,POINT (-82.288889 31.375278)
4,ABAT2,no_flooding,Albany,32.732778,-99.140278,Hubbard Creek,TX,3.40,2017-09-08 21:00:00,ft,...,43.00,,ft,sjt,NAD83/WGS84,HGIRG,-999.00,kcfs,https://water.weather.gov/ahps2/hydrograph.php...,POINT (-99.140278 32.732778)


Since we're only interested in the gauges which are flooding, we'll select those. 

In [14]:
flood = riverdata[riverdata.Status.isin(['low_threshold', 
                                         'action',
                                         'minor',
                                         'moderate',
                                         'major'])]

print("%d stations with some flooding (types: %s)" % (len(flood), set(flood.Status)))

82 stations with some flooding (types: {'major', 'moderate', 'action', 'minor', 'low_threshold'})


Now comes the interesting part. We'll create a folium map with Stamen Terrain tiles. Then, for each for the rows in the `flood` GeoDataFrame, we'll choose an appropriate color for the marker on the map based on the flood stage, and plot that marker given the data in the DataFrame. 

In [15]:
rivermap = folium.Map(location=[39.8283, -98.5795], tiles='Stamen Terrain', zoom_start=4)
for index, row in flood.iterrows():
    iconcolor='blue'
    if row['Status'] == "major":
        iconcolor = 'darkred'
    elif row['Status'] == 'action':
        iconcolor = 'lightred'
    marker = folium.Marker((row['Latitude'],row['Longitude']), popup="%s:%s" % (row['GaugeLID'], row['Status']),
                           icon=folium.Icon(color=iconcolor, icon='warning-sign'))
    marker.add_to(rivermap)
rivermap

There you have it! A real-time view of the river gauges in the United States which are flooding right now. Clicking on each of the markers brings up a popup with the Gauge ID and the flood descriptor. Try it!

But we can take this one step further. What if we wanted to bring the real-time hydrograph images for each of those points into the map? We can do that by *embedding HTML into the popups for each marker*. We do this by defining the HTML template for each marker, and using Python string completion to fill in the gaps for each data point. Then we can add that HTML to the markers as a Popup object. 

In [16]:
popupHTMLTemplate = "<img src='http://water.weather.gov/resources/hydrographs/%s_hg.png' width=500> <a href='%s' target='_blank'>Click Here for More</a>"

rivermap = folium.Map(location=[39.8283, -98.5795], tiles='Stamen Terrain', zoom_start=4)

for index, row in flood.iterrows():
    iconcolor='blue'
    if row['Status'] == "major":
        iconcolor = 'darkred'
    elif row['Status'] == 'action':
        iconcolor = 'lightred'
        
    completePopup = popupHTMLTemplate % (row["GaugeLID"].lower(), row["URL"])
    pu_iframe = folium.IFrame(html=completePopup, width=530, height=425)
    popup = folium.Popup(pu_iframe, max_width=2650)
    marker = folium.Marker((row['Latitude'],row['Longitude']), popup=popup,
                           icon=folium.Icon(color=iconcolor, icon='warning-sign'))
    marker.add_to(rivermap)
rivermap.save("rivermap.html")
rivermap

View your map full-screen [here](rivermap.html).

## Choropleth Maps
One of the most common ways to visualize geospatial data, especially with data that is partitioned geographically like survey or census data, is the **choropleth map**. The word "choropleth" comes from Greek: χῶρος ("area/region") + πλῆθος ("multitude"), and is basically a thematic map which uses color to distinguish between data points separated by geographic area. Folium is good at making these, so we'll make one here. 

In Folium, the philosophy behind the choropleth is that the Folium library joins some geospatial data (in GeoJSON) format to some tabular data (as a Pandas dataframe) on some pre-defined key, and plots a specified column from the tabular data on the map. 

For our example, we'll use the US Department of the Treasury's "[Brewery Count by State](https://catalog.data.gov/dataset/brewery-count-by-state-1984-march-31-2017)" dataset which I've cleaned  to be easy to use.

In [17]:
import pandas as pd
breweries = pd.read_csv("../data/breweries.csv")
breweries.head()

,STATE,COUNT
0,AK,40
1,AL,37
2,AR,36
3,AZ,120
4,CA,981


We'll join this data to the [shapefiles](https://www.census.gov/geo/maps-data/data/cbf/cbf_state.html) provided by the US Census Bureau.

In [18]:
! unzip -o ../data/cb_2016_us_state_5m.zip

Archive:  ../data/cb_2016_us_state_5m.zip
  inflating: cb_2016_us_state_5m.shp.ea.iso.xml  
  inflating: cb_2016_us_state_5m.shp.iso.xml  
  inflating: cb_2016_us_state_5m.shp.xml  
  inflating: cb_2016_us_state_5m.shp  
  inflating: cb_2016_us_state_5m.shx  
  inflating: cb_2016_us_state_5m.dbf  
  inflating: cb_2016_us_state_5m.prj  
 extracting: cb_2016_us_state_5m.cpg  


In [19]:
states = gpd.read_file("cb_2016_us_state_5m.shp")
states.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,01,01779775,0400000US01,01,AL,Alabama,00,131173688951,4593686489,"(POLYGON ((-88.04374299999999 30.517423, -88.0..."
1,02,01785533,0400000US02,02,AK,Alaska,00,1477946266785,245390495931,"(POLYGON ((-133.655819 55.625617, -133.624921 ..."
2,04,01779777,0400000US04,04,AZ,Arizona,00,294198560125,1027346486,"POLYGON ((-114.799683 32.593621, -114.809393 3..."
3,08,01779779,0400000US08,08,CO,Colorado,00,268429343790,1175112870,"POLYGON ((-109.060253 38.599328, -109.059541 3..."
4,09,01779780,0400000US09,09,CT,Connecticut,00,12542638347,1815476291,"POLYGON ((-73.72777499999999 41.100696, -73.69..."


Looking at our data, we can see that the **"COUNT"** column in the breweries file contains our plottable value, the **"STATE"** column and the **STUSPS** column in the breweries and states file respectively are our keys between data and geometry. We express this in folium as follows:

In [20]:
brewmap = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
brewmap.choropleth(data=breweries,
                   geo_data=states, 
                   columns = ['STATE', 'COUNT'], # specifies [<key>, <value>] columns in breweries dataframe
                   key_on  = 'feature.properties.STUSPS',  # specifies <key> in states GeoJSON Shapefile, 
                   fill_color='BuPu', fill_opacity=0.9, line_opacity=0.5,
                   threshold_scale = list(range(0, 1200, 200)),
                   legend_name = "Number of Breweries",
                   highlight=True)
brewmap.save("breweries.html")
iframe("breweries.html")

In `breweries.html` there's the map which we just built. In the `brewmap.choropleth` call, we can see that:

* `data`: is our dataframe of **STATE**:**COUNT** data
* `geo_data` is in this case a GeoPandas geodataframe containing the geometries (and other info) of U.S. States from the Census shapefile
* `columns` is a length-2 list containing the [key, value] columns in the tabular data, where the key references a geometry in the geodataframe and the value is the plottable value for the choropleth. 
* `key_on` is that corresponding key in the GeoJSON representation of the geodataframe, which when generated from Geopandas is of the format "feature.properties.*key_column_name*"
* `fill_color` is a ColorBrewer palette name (see them [here](http://www.datavis.ca/sasmac/brewer.all.jpg))
* `threshold_scale` is a list which specifies where to add breakpoints along the color scale
* `legend_name` is text which appears below the choropleth legend. 

It is beyond the scope of this tutorial to add pop-ups to this choropleth map, as it requires manually adding features (as above, for the rivermap), but is not a large leap conceptually from what has been learned here. 

## Conclusion 
There you have it, a tutorial on Folium. We've covered:

* How to begin and save a simple Folium map in Python
* Basics of markers and polygons
* How to interface with shapefiles and other geospatial data using Geopandas and GeoJson
* Different basemaps
* Practicing synthesizing these skills with real data and producing customizable HTML popups. 
* Creating a choropleth map with Folium. 

There are, of course, many other things that Folium can do. Fortunately there are some good examples and tutorials (+ documentation!) for the library on the Github page, so if you're interested in learning more that's where you should go. 